## Importing necessary libraries

In [ ]:
#!pip install random-user-agent

In [1]:
from selenium import webdriver

import time
import pandas as pd

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import numpy as np

## Trying to not get detected

In [2]:
software_names=[SoftwareName.CHROME.value]
operating_systems=[OperatingSystem.WINDOWS.value]

user_agent_rotator=UserAgent(software_names=software_names,
                            operating_systems=operating_systems,
                             limit=100)
user_agent=user_agent_rotator.get_random_user_agent()

options = webdriver.ChromeOptions() 

options.add_argument(f'user-agent={user_agent}')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

## Scraping

In [ ]:
# Passing the path of the webdriver
PATH = "C:\\Program Files (x86)\\chromedriver.exe"

driver = webdriver.Chrome(options=options, executable_path=PATH)

# Opening the link
driver.get("https://www.walmart.com/ip/Clorox-Disinfecting-Wipes-225-Count-Value-Pack-Crisp-Lemon-and-Fresh-Scent-3-Pack-75-Count-Each/14898365")

time.sleep(3)

# Clicking on 'See all reviews'
link = driver.find_element_by_css_selector("#customer-reviews-header > div:nth-child(2) > div > div.ReviewsBar-link-buttons > a.button.ReviewBtn-container.ReviewsHeaderWYR-seeAll.button--primary")
link.click()

time.sleep(3)

# Sorting the reviews from newest to oldest
link = driver.find_element_by_css_selector("body > div.js-content > div > div > div > div.page-content-wrapper > div > div.frequent-mentions-container > div > div.frequent-mentions-results-bar > div > div.reviews-sort-bar > div > div.reviews-sort-bar-sort-field.field.field--select.field--compact > select > option:nth-child(3)")
link.click()
time.sleep(3)

# Creating lists to store the scraped data
dates=[]
names=[]
titles=[]
descriptions=[]
ratings=[]

# Setting condition to True
CONDITION=True

# Running the while loop until the 'CONDITION' becomes False
while CONDITION:
     # Getting all the elements which are present in the 'customer-review-body' div which has all the
     # titles, dates, name, description and rating
    iter=driver.find_elements_by_css_selector('div.Grid-col.customer-review-body')
    
    # For each review-
    for i in iter:
        # Some of the values were not present in the website. To avoid getting no values,
        # we'll use exceptions, when raised, will result in Nan's getting appended rather than nothing

        print("____________________________________________________________________________________")
        # Getting the date
        try:
            date = i.find_element_by_css_selector('span.review-date-submissionTime').text
            temp = date[-4:]
            # Printing the dates just to keep an eye on the scraping
            print("YEAR:",date)
            dates.append(date)
            # Changing the status of 'CONDITION' to False if the year gets to '2020'
            # which will break the while loop
            if(int(temp)<2021):
                CONDITION = False
                break
        except Exception as e:
            print(e)

        # Getting the title
        try:
            title = i.find_element_by_css_selector('h3.review-title.font-bold').text
            titles.append(title)
        except Exception as E:
            titles.append(np.nan)
            
        # Getting the name
        try:
            name = i.find_element_by_css_selector('span.review-footer-userNickname').text
            names.append(name)

        except Exception as E:
            names.append(np.nan)

        # Getting the description
        try:
            description = i.find_element_by_css_selector('div.review-description').text
            descriptions.append(description)
        except Exception as E:
            descriptions.append(np.nan)

        # Getting the rating
        try:
            rating = i.find_element_by_css_selector('span.visuallyhidden.seo-avg-rating').text
            ratings.append(rating)
        except Exception as E:
            ratings.append(np.nan)
            
        time.sleep(1)

            
    time.sleep(2)       
    # Clicking on the next page       
    link = driver.find_element_by_css_selector("button.paginator-btn.paginator-btn-next")
    link.click()

## Creating dataframe

In [6]:
df = pd.DataFrame(list(zip(names,dates,titles,ratings,descriptions)), columns=['Name','Date','Title','Rating','Description'])

In [7]:
df.head()

,Name,Date,Title,Rating,Description
0,Goodeat,"August 3, 2021",NaN,2.0,One plastic container was crunched and leaked ...
1,Vicki,"August 3, 2021","Clorox Disinfecting Wipes,(225 Count Pack)",1.0,The box this item was delivered in was wet!! W...
2,Mark,"August 3, 2021",Thank you!,5.0,This is good price and great product!
3,James,"August 2, 2021",NaN,2.0,Whatever liquid thats not absorbed in the towe...
4,Ro,"August 1, 2021",NaN,5.0,"Love these, very easy to use"


In [8]:
df['Date'] = pd.to_datetime(df.Date,format="%B %d, %Y")

In [9]:
df = df.sort_values(by="Date",ascending=False)

In [10]:
df.head()

,Name,Date,Title,Rating,Description
0,Goodeat,2021-08-03,NaN,2.0,One plastic container was crunched and leaked ...
20,Goodeat,2021-08-03,NaN,2.0,One plastic container was crunched and leaked ...
21,Vicki,2021-08-03,"Clorox Disinfecting Wipes,(225 Count Pack)",1.0,The box this item was delivered in was wet!! W...
1,Vicki,2021-08-03,"Clorox Disinfecting Wipes,(225 Count Pack)",1.0,The box this item was delivered in was wet!! W...
2,Mark,2021-08-03,Thank you!,5.0,This is good price and great product!


## Exporting data into csv

In [ ]:
df.to_csv('Reiviews.csv',index=False)